In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
from pyspark.sql.types import StructField, StructType, StructType, IntegerType, FloatType, DateType

from dotenv import load_dotenv
import sys
import os
import datetime
# Добавляем корневую директорию проекта в sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

load_dotenv()

spark = (SparkSession.builder
         .appName("cbr_currency_etl")
         .getOrCreate())

spark

KeyboardInterrupt: 

In [4]:
from consultant.extractors.base import get_cbr_currency_rate_json



currency_list = list(get_cbr_currency_rate_json()['Valute'].values())

raw_currency = spark.createDataFrame(currency_list)
raw_currency.show(100, truncate=False)

/home/b.kustov/.local/lib/python3.8/site-packages/pyspark/sql/session.py:381: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+--------+-------+----------------------------------------+-------+-------+--------+--------+
|CharCode|ID     |Name                                    |Nominal|NumCode|Previous|Value   |
+--------+-------+----------------------------------------+-------+-------+--------+--------+
|AUD     |R01010 |Австралийский доллар                    |1      |036    |65.6273 |65.4655 |
|AZN     |R01020A|Азербайджанский манат                   |1      |944    |60.5368 |60.587  |
|GBP     |R01035 |Фунт стерлингов Соединенного королевства|1      |826    |129.8138|130.7146|
|AMD     |R01060 |Армянских драмов                        |100    |051    |26.1319 |26.1025 |
|BYN     |R01090B|Белорусский рубль                       |1      |933    |29.6937 |29.7415 |
|BGN     |R01100 |Болгарский лев                          |1      |975    |55.3439 |55.2846 |
|BRL     |R01115 |Бразильский реал                        |1      |986    |17.0393 |17.0226 |
|HUF     |R01135 |Венгерских форинтов                     |1

In [5]:
prepared_currency = raw_currency \
    .withColumn('VunitRate', col('Value') / col('Nominal')) \
    .withColumn('ValCursDate', lit(datetime.date.today())) \
    .select(['CharCode', 'Nominal', 'Value', 'VunitRate', 'ValCursDate'])

new_schema = StructType([
    StructField('code_iso', StringType(), False),
    StructField('nominal', IntegerType(), False),
    StructField('rate', FloatType(), False),
    StructField('unit_rate', FloatType(), False),
    StructField('on_date', DateType(), False)
])
prepared_currency 
prepared_currency.show(100, truncate=False)

+--------+-------+--------+---------------------+-----------+
|CharCode|Nominal|Value   |VunitRate            |ValCursDate|
+--------+-------+--------+---------------------+-----------+
|AUD     |1      |65.4655 |65.4655              |2024-12-17 |
|AZN     |1      |60.587  |60.587               |2024-12-17 |
|GBP     |1      |130.7146|130.7146             |2024-12-17 |
|AMD     |100    |26.1025 |0.261025             |2024-12-17 |
|BYN     |1      |29.7415 |29.7415              |2024-12-17 |
|BGN     |1      |55.2846 |55.2846              |2024-12-17 |
|BRL     |1      |17.0226 |17.0226              |2024-12-17 |
|HUF     |100    |26.405  |0.26405              |2024-12-17 |
|VND     |10000  |42.4384 |0.0042438400000000005|2024-12-17 |
|HKD     |1      |13.2798 |13.2798              |2024-12-17 |
|GEL     |1      |36.576  |36.576               |2024-12-17 |
|DKK     |1      |14.4965 |14.4965              |2024-12-17 |
|AED     |1      |28.0457 |28.0457              |2024-12-17 |
|USD    

In [6]:
# Writing

from consultant.tools.basic import get_jdbc_url_for_gp, get_properies_for_gp

# import importlib
# import consultant.tools.basic as module

# importlib.reload(module)

print(get_properies_for_gp())
print(get_jdbc_url_for_gp())

prepared_currency.write.jdbc(
    url=get_jdbc_url_for_gp(),
    table='b_kustov.currency_rate',
    properties=get_properies_for_gp(),
    mode='append'
)

{'user': 'wave12_user_a5', 'password': 'pass', 'driver': 'org.postgresql.Driver'}
jdbc:postgresql://172.17.1.32:5432/wave12_team_a


Py4JJavaError: An error occurred while calling o192.jdbc.
: org.postgresql.util.PSQLException: ERROR: relation "currency_rate" already exists
	at org.postgresql.core.v3.QueryExecutorImpl.receiveErrorResponse(QueryExecutorImpl.java:2712)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2400)
	at org.postgresql.core.v3.QueryExecutorImpl.execute(QueryExecutorImpl.java:367)
	at org.postgresql.jdbc.PgStatement.executeInternal(PgStatement.java:498)
	at org.postgresql.jdbc.PgStatement.execute(PgStatement.java:415)
	at org.postgresql.jdbc.PgStatement.executeWithFlags(PgStatement.java:335)
	at org.postgresql.jdbc.PgStatement.executeCachedSql(PgStatement.java:321)
	at org.postgresql.jdbc.PgStatement.executeWithFlags(PgStatement.java:297)
	at org.postgresql.jdbc.PgStatement.executeUpdate(PgStatement.java:270)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.createTable(JdbcUtils.scala:894)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRela

In [5]:
spark.stop()